In [ ]:
#obiettivo: capire come sfruttare questo scraping per creare le liste che poi ti permettono di fare il dataframe

In [ ]:
#SCRAPING MARIA ANTONIAK

In [16]:
import argparse
from collections import Counter
from datetime import datetime
import json
import os
import regex as re
import time
import bs4
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException, ElementNotVisibleException, StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from urllib.request import urlopen
from urllib.error import HTTPError
import credentials
from selenium.webdriver.common.by import By
import pandas as pd
from chromedriver_py import binary_path 
import geckodriver_autoinstaller
PATH='C:\\Users\\aless\\OneDrive\\Desktop\\Laurea_magistrale\\Tesi\\chromedriver.exe'
driver = webdriver.Chrome(PATH)

In [4]:

import pandas as pd
meta_en=pd.read_csv("C:\\Users\\aless\\OneDrive\\Desktop\\Laurea_magistrale\\Tesi\\Dati\\Dati_English\\metadati_EN.csv")
meta_en



,Unnamed: 0,index,Genere,Genere_Suggerito,Pud_Data,Title,Author,ISBN,Num_Pages_GR,Stars_GR,Distr_Rat_GR,N_Votes_GR,N_Reviews_GR,GR_link,Stars_AZ,Distr_Rat_AZ,N_Votes_AZ,N_Reviews_AZ,AZ_link
0,0,0,Children_Lit,"['Childrens\n4,244 users', 'Childrens > Pictur...",Published 2000 by Red Fox (first published Apr...,Where the Wild Things Are,Maurice Sendak,9780099408390,38,4.22,"['53%', '25%', '14%', '5%', '3%']",884675,12480,https://www.goodreads.com/book/show/19543.Wher...,4.9,"['92%', '5%', '2%', '1%', '1%']",23220.0,3003.0,https://www.amazon.it/s?k=9780099408390&i=stri...
1,1,1,Children_Lit,"['Childrens\n4,125 users', 'Childrens > Pictur...",Published October 7th 1964 by HarperCollins Pu...,The Giving Tree,Shel Silverstein,9780060256654,64,4.37,"['61%', '22%', '10%', '3%', '3%']",978948,19120,https://www.goodreads.com/book/show/370493.The...,4.9,"['92%', '5%', '2%', '1%', '1%']",18132.0,3463.0,https://www.amazon.it/s?k=9780060256654&i=stri...
2,2,2,Children_Lit,"['Childrens\n3,957 users', 'Fiction\n3,617 use...",Published June 1st 1998 by Puffin Books (first...,Matilda,Roald Dahl,9780141301068,240,4.32,"['53%', '31%', '13%', '2%', '1%']",741229,18376,https://www.goodreads.com/book/show/39988.Matilda,4.8,"['86%', '9%', '3%', '1%', '1%']",5422.0,1160.0,https://www.amazon.it/s?k=9780141301068&i=stri...
3,3,3,Children_Lit,"['Classics\n18,253 users', 'Fiction\n5,704 use...",Published September 1st 1998 by Children's Cla...,The Secret Garden,Frances Hodgson Burnett,9780517189603,331,4.13,"['43%', '34%', '17%', '4%', '2%']",984296,21409,https://www.goodreads.com/book/show/2998.The_S...,4.6,"['78%', '13%', '5%', '2%', '3%']",8946.0,2478.0,https://www.amazon.it/s?k=9780517189603&i=stri...
4,4,4,Children_Lit,"['Childrens\n3,071 users', 'Fantasy\n2,796 use...",Published September 10th 2002 by Alfred A. Kno...,James and the Giant Peach,Roald Dahl,9780375814242,146,4.01,"['38%', '34%', '21%', '5%', '2%']",414915,7540,https://www.goodreads.com/book/show/6689.James...,4.8,"['85%', '9%', '3%', '1%', '2%']",4101.0,838.0,https://www.amazon.it/s?k=9780375814242&i=stri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1063,Scify,"['Science Fiction\n1,076 users', 'Fiction\n508...",Published December 1997 by Spectra/Bantam Book...,The Postman,David Brin,9780553278743,321,3.90,"['29%', '40%', '24%', '5%', '2%']",34029,1387,https://www.goodreads.com/book/show/889284.The...,4.4,"['64%', '22%', '9%', '3%', '2%']",670.0,266.0,https://www.amazon.it/s?k=9780553278743&i=stri...
996,996,1064,Scify,"['Science Fiction\n1,069 users', 'Fiction\n357...",Published January 15th 2000 by Tor Science Fic...,A Deepness in the Sky,Vernor Vinge,9780812536355,775,4.30,"['50%', '34%', '12%', '3%', '1%']",31799,1075,https://www.goodreads.com/book/show/226004.A_D...,4.6,"['72%', '18%', '6%', '3%', '1%']",458.0,228.0,https://www.amazon.it/s?k=9780812536355&i=stri...
997,997,1065,Scify,"['Science Fiction\n1,066 users', 'Fiction\n1,0...",Published December 5th 2005 by HarperCollins P...,That Hideous Strength,C.S. Lewis,9780007157174,534,3.88,"['34%', '33%', '23%', '7%', '3%']",34925,2091,https://www.goodreads.com/book/show/100933.Tha...,4.7,"['79%', '12%', '5%', '2%', '1%']",800.0,320.0,https://www.amazon.it/s?k=9780007157174&i=stri...
998,998,1066,Scify,"['Science Fiction\n1,065 users', 'Fiction\n410...",Published July 7th 2015 by Orbit,Aurora,Kim Stanley Robinson,9780316098106,466,3.76,"['25%', '40%', '24%', '8%', '3%']",23091,2709,https://www.goodreads.com/book/show/23197269-a...,3.9,"['40%', '29%', '16%', '9%', '5%']",1353.0,769.0,https://www.amazon.it/s?k=9780316098106&i=stri...


In [5]:
isbn=meta_en['ISBN'].to_list()
title=meta_en['Title'].to_list()
gen=meta_en['Genere'].to_list()
#mylinks=meta_en['GR_link'].to_list()
print(isbn[:10])
print(title[:10])
print(gen[:10])
#print(mylinks[:10])

[9780099408390, 9780060256654, 9780141301068, 9780517189603, 9780375814242, 9780241003008, 9780141311371, 9780060775858, 9780590032490, 9780439206471]
['Where the Wild Things Are', 'The Giving Tree', 'Matilda', 'The Secret Garden', 'James and the Giant Peach', 'The Very Hungry Caterpillar', 'The BFG', 'Goodnight Moon', 'The Witches', 'The Bad Beginning']
['Children_Lit', 'Children_Lit', 'Children_Lit', 'Children_Lit', 'Children_Lit', 'Children_Lit', 'Children_Lit', 'Children_Lit', 'Children_Lit', 'Children_Lit']


In [6]:
#creiamo la colonna di bookid
bookid_list=[]
listid=meta_en['GR_link'].to_list()
for el in listid:
    elem=el.split('.')
    ids=re.findall(r'\d+' ,elem[2])[0]
    bookid_list.append(ids)
bookid_list[:10]
    

['19543',
 '370493',
 '39988',
 '2998',
 '6689',
 '4948',
 '6319',
 '32929',
 '6327',
 '78411']

In [7]:
len(bookid_list)

1000

In [8]:


RATING_STARS_DICT = {'it was amazing': 5,
                     'really liked it': 4,
                     'liked it': 3,
                     'it was ok': 2,
                     'did not like it': 1,
                     '': None}


def switch_reviews_mode(driver, book_id, sort_order, rating=''):
    """
    Copyright (C) 2019 by Omar Einea: https://github.com/OmarEinea/GoodReadsScraper
    Licensed under GPL v3.0: https://github.com/OmarEinea/GoodReadsScraper/blob/master/LICENSE.md
    Accessed on 2019-12-01.
    """
    edition_reviews=False
    driver.execute_script(
        'document.getElementById("reviews").insertAdjacentHTML("beforeend", \'<a data-remote="true" rel="nofollow"'
        f'class="actionLinkLite loadingLink" data-keep-on-success="true" id="switch{rating}{sort_order}"' +
        f'href="/book/reviews/{book_id}?rating={rating}&sort={sort_order}' +
        ('&edition_reviews=true' if edition_reviews else '') + '">Switch Mode</a>\');' +
        f'document.getElementById("switch{rating}{sort_order}").click()'
    )
    return True


def get_rating(node):
    if len(node.find_all('span', {'class': 'staticStars'})) > 0:
        rating = node.find_all('span', {'class': 'staticStars'})[0]['title']
        return RATING_STARS_DICT[rating]
    return ''


def get_user_name(node):
    if len(node.find_all('a', {'class': 'user'})) > 0:
        return node.find_all('a', {'class': 'user'})[0]['title']
    return ''

def get_user_url(node):
    if len(node.find_all('a', {'class': 'user'})) > 0:
        return node.find_all('a', {'class': 'user'})[0]['href']
    return ''


def get_date(node):
    if len(node.find_all('a', {'class': 'reviewDate createdAt right'})) > 0:
        return node.find_all('a', {'class': 'reviewDate createdAt right'})[0].text
    return ''


def get_text(node):

    display_text = ''
    full_text = ''

    if len(node.find_all('span', {'class': 'readable'})) > 0:
        for child in node.find_all('span', {'class': 'readable'})[0].children:
            if child.name == 'span' and 'style' not in child:
                display_text = child.text
            if child.name == 'span' and 'style' in child and child['style'] == 'display:none':
                full_text = child.text

    if full_text:
        return full_text

    return display_text


def get_num_likes(node):
    if node.find('span', {'class': 'likesCount'}) and len(node.find('span', {'class': 'likesCount'})) > 0:
        likes = node.find('span', {'class': 'likesCount'}).text
        if 'likes' in likes:
            return int(likes.split()[0])
    return 0


def get_shelves(node): 
    shelves = []       
    if node.find('div', {'class': 'uitext greyText bookshelves'}):
        _shelves_node = node.find('div', {'class': 'uitext greyText bookshelves'})
        for _shelf_node in _shelves_node.find_all('a'):
            shelves.append(_shelf_node.text)
    return shelves

def get_id(bookid):
    pattern = re.compile("([^.]+)")
    return pattern.search(bookid).group()

    
def scrape_reviews_on_current_page(driver, url, book_id, sort_order,number):

    reviews = []

    # Pull the page source, load into BeautifulSoup, and find all review nodes.
    source = driver.page_source
    soup = bs4.BeautifulSoup(source, 'lxml')
    nodes = soup.find_all('div', {'class': 'review'})
    #book_title = soup.find(id='bookTitle').text.strip()
    
    # Iterate through and parse the reviews.
    for node in nodes:
        review_id = re.search('[0-9]+', node['id']).group(0)
        reviews.append({#'book_id_title': book_id,
                        #'book_id': get_id(book_id),
                        'Genere': gen[number],
                        'Titolo': title[number],
                        'ISBN': isbn[number],
                        #'review_url': f"https://www.goodreads.com/review/show/{review_id}", 
                        'review_id': review_id, #poi la elimini dal dataframe, serve solo per il controllo duplicati
                        'Rev_Date_GR': get_date(node), 
                        'Rating_GR': get_rating(node), 
                        'Userlist_GR': get_user_name(node),
                        #'user_url': get_user_url(node), 
                        'N_Likes_GR': get_num_likes(node),
                        #'sort_order': sort_order,
                        'Shelves': get_shelves(node),
                        'Rev_Content_GR': get_text(node)})

    return reviews


def check_for_duplicates(reviews):
    review_ids = [r['review_id'] for r in reviews]  
    num_duplicates = len([_id for _id, _count in Counter(review_ids).items() if _count > 1])
    return num_duplicates


def get_reviews_first_ten_pages(driver, book_id, sort_order,number):

    reviews = []
    url = 'https://www.goodreads.com/book/show/' + book_id
    try:
        driver.get(url)
    except:
        time.sleep(3)
        driver.get(url)

    try:
        source = driver.page_source
    except:
        time.sleep(3)
        source = driver.page_source

    try:
        time.sleep(4)
        if sort_order != 'default':
            switch_reviews_mode(driver, book_id, sort_order)
            time.sleep(2)

        if sort_order == 'newest' or sort_order == 'oldest':
            select = Select(driver.find_element_by_name('language_code'))
            select.select_by_value('en')
            time.sleep(4)
    

    
        # Scrape the first page of reviews.
        reviews = scrape_reviews_on_current_page(driver, url, book_id, sort_order,number)
        #print('Scraped page 1')
        # GoodReads will only load the first 10 pages of reviews.
        # Click through each of the following nine pages and scrape each page.
        page_counter = 2
        while page_counter <=10:
            try:
                if driver.find_element_by_link_text(str(page_counter)):
                    driver.find_element_by_link_text(str(page_counter)).click()
                    time.sleep(3)
                    reviews += scrape_reviews_on_current_page(driver, url, book_id, sort_order,number)
                    #print(f"Scraped page {page_counter}")
                    page_counter += 1
                else:
                    return reviews
            
            except NoSuchElementException:
                if page_counter == 10:
                    try:
                        driver.find_element_by_link_text(str(9)).click()
                        time.sleep(2)
                        continue
                    except:
                        return reviews
                else:
                    try:
                        if driver.find_element(By.CLASS_NAME, 'next_page'):
                            driver.find_element(By.CLASS_NAME, 'next_page').click()
                            continue
                    except NoSuchElementException:
                        #print("'next' button not found either")
                        return reviews
            
            except ElementNotVisibleException:
                #print('ERROR ElementNotVisibleException: Pop-up detected, reloading the page.')
                reviews = get_reviews_first_ten_pages(driver, book_id, sort_order,number)
                return reviews
                        
            except ElementClickInterceptedException:
                #print(f'🚨 ElementClickInterceptedException (Likely a pop-up)🚨\n🔄 Refreshing Goodreads site and skipping problem page {page_counter}🔄')
                driver.get(url)
                time.sleep(3)
                page_counter += 1
                continue
                
            except StaleElementReferenceException:
                #print('ERROR: StaleElementReferenceException\nRefreshing Goodreads site and skipping problem page {page_counter} ')
                driver.get(url)
                time.sleep(3)
                page_counter += 1
                continue
                
    except ElementClickInterceptedException:
                #print(f'🚨 ElementClickInterceptedException (Likely a pop-up)🚨\n🔄 Refreshing Goodreads site and rescraping book🔄')
                driver.get(url)
                time.sleep(3)
                reviews = get_reviews_first_ten_pages(driver, book_id, sort_order,number)
                return reviews
                
    except ElementNotInteractableException:
            #print('🚨 ElementNotInteractableException🚨 \n🔄 Refreshing Goodreads site and rescraping book🔄')
            reviews = get_reviews_first_ten_pages(driver, book_id, sort_order,number)
            return reviews

        
    if check_for_duplicates(reviews) >= 30:
        print(f'ERROR: {check_for_duplicates(reviews)} duplicates found! Re-scraping this book.')
        reviews = get_reviews_first_ten_pages(driver, book_id, sort_order,number)
        return reviews
    else:
        return reviews

    return reviews
    


In [41]:
#goodreads=[]
number=794 #[704,794,939] sono da riscrapre nel caso
#sort_order='newest'
for el in bookid_list[795:850]:
    number+=1
    try:
        try:
            rev=get_reviews_first_ten_pages(driver, el, 'newest',number)
        except NoSuchElementException:
            time.sleep(60*2)
            rev=get_reviews_first_ten_pages(driver, el, 'newest',number)
    except NoSuchElementException:
        time.sleep(60*3)
        rev=get_reviews_first_ten_pages(driver, el, 'newest',number)
        
    print(number)
    print(len(rev))
    print('\n')
    for el in rev:
        goodreads.append(el)
goodreads_data=pd.DataFrame(goodreads)
goodreads_data

795
240


796
180


797
300


798
270


799
240


800
180


801
210


802
210


803
210


804
180


805
240


806
300


807
180


808
300


809
150


810
210


811
180


812
300


813
180


814
180


815
180


816
240


817
210


818
180


819
180


820
180


821
180


822
179


823
270


824
180


825
240


826
180


827
210


828
300


829
150


830
180


831
180


832
210


833
300


834
300


835
240


836
270


837
150


838
180


839
210


840
240


841
180


842
180


843
210


844
300


845
180


846
210


847
210


848
180


849
300




,Genere,Titolo,ISBN,review_id,Rev_Date_GR,Rating_GR,Userlist_GR,N_Likes_GR,Shelves,Rev_Content_GR
0,Thriller,Triptych,9780385339469,4244899561,"Oct 09, 2021",4,Carolyn Taylor,0,[],Looking to be another (older) but good series ...
1,Thriller,Triptych,9780385339469,4276267692,"Oct 07, 2021",5,soperedi,0,[],"Good thrillerI very liked this book, fascinati..."
2,Thriller,Triptych,9780385339469,3715245046,"Oct 04, 2021",4,Ellen,0,"[library, audiobook]",I’ve been wanting to start this series for som...
3,Thriller,Triptych,9780385339469,4271309293,"Oct 04, 2021",5,Harry Fuhrman,0,[],good read hooks you
4,Thriller,Triptych,9780385339469,4271028087,"Oct 04, 2021",2,Yumi R,0,[],Predictable. Hoping the storyline would pick u...
...,...,...,...,...,...,...,...,...,...,...
31787,Scify,Golden Son,9780345539823,4091633148,"Jul 02, 2021",5,Caleb Barker,0,[],Wow just really elevated what was established ...
31788,Scify,Golden Son,9780345539823,4074181907,"Jul 02, 2021",4,Jolien,2,[],This book became better and better the further...
31789,Scify,Golden Son,9780345539823,4067603858,"Jul 02, 2021",3,Kayla K,0,[],"I liked this book better than the first one, b..."
31790,Scify,Golden Son,9780345539823,4061442045,"Jul 01, 2021",5,Farid Yuzma Wiarco,0,[own],Gorydamn packed


In [42]:
colonne=goodreads_data.columns.to_list()
#print(colonne)
gr_data=goodreads_data.drop_duplicates(subset=colonne[3],keep='first',inplace=False)
gr_data

,Genere,Titolo,ISBN,review_id,Rev_Date_GR,Rating_GR,Userlist_GR,N_Likes_GR,Shelves,Rev_Content_GR
0,Thriller,Triptych,9780385339469,4244899561,"Oct 09, 2021",4,Carolyn Taylor,0,[],Looking to be another (older) but good series ...
1,Thriller,Triptych,9780385339469,4276267692,"Oct 07, 2021",5,soperedi,0,[],"Good thrillerI very liked this book, fascinati..."
2,Thriller,Triptych,9780385339469,3715245046,"Oct 04, 2021",4,Ellen,0,"[library, audiobook]",I’ve been wanting to start this series for som...
3,Thriller,Triptych,9780385339469,4271309293,"Oct 04, 2021",5,Harry Fuhrman,0,[],good read hooks you
4,Thriller,Triptych,9780385339469,4271028087,"Oct 04, 2021",2,Yumi R,0,[],Predictable. Hoping the storyline would pick u...
...,...,...,...,...,...,...,...,...,...,...
31787,Scify,Golden Son,9780345539823,4091633148,"Jul 02, 2021",5,Caleb Barker,0,[],Wow just really elevated what was established ...
31788,Scify,Golden Son,9780345539823,4074181907,"Jul 02, 2021",4,Jolien,2,[],This book became better and better the further...
31789,Scify,Golden Son,9780345539823,4067603858,"Jul 02, 2021",3,Kayla K,0,[],"I liked this book better than the first one, b..."
31790,Scify,Golden Son,9780345539823,4061442045,"Jul 01, 2021",5,Farid Yuzma Wiarco,0,[own],Gorydamn packed


In [43]:
gr_data=gr_data.drop('review_id',axis=1) #e altre se serve
gr_data

,Genere,Titolo,ISBN,Rev_Date_GR,Rating_GR,Userlist_GR,N_Likes_GR,Shelves,Rev_Content_GR
0,Thriller,Triptych,9780385339469,"Oct 09, 2021",4,Carolyn Taylor,0,[],Looking to be another (older) but good series ...
1,Thriller,Triptych,9780385339469,"Oct 07, 2021",5,soperedi,0,[],"Good thrillerI very liked this book, fascinati..."
2,Thriller,Triptych,9780385339469,"Oct 04, 2021",4,Ellen,0,"[library, audiobook]",I’ve been wanting to start this series for som...
3,Thriller,Triptych,9780385339469,"Oct 04, 2021",5,Harry Fuhrman,0,[],good read hooks you
4,Thriller,Triptych,9780385339469,"Oct 04, 2021",2,Yumi R,0,[],Predictable. Hoping the storyline would pick u...
...,...,...,...,...,...,...,...,...,...
31787,Scify,Golden Son,9780345539823,"Jul 02, 2021",5,Caleb Barker,0,[],Wow just really elevated what was established ...
31788,Scify,Golden Son,9780345539823,"Jul 02, 2021",4,Jolien,2,[],This book became better and better the further...
31789,Scify,Golden Son,9780345539823,"Jul 02, 2021",3,Kayla K,0,[],"I liked this book better than the first one, b..."
31790,Scify,Golden Son,9780345539823,"Jul 01, 2021",5,Farid Yuzma Wiarco,0,[own],Gorydamn packed


In [44]:
from langdetect import detect
lang=[]
for index, row in gr_data['Rev_Content_GR'].iteritems():
    try:
        language = detect(row) #detecting each row
        lang.append(language)
    except:
        lang.append('not found')
lang[:10]

['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en']

In [45]:
gr_data['Language_GR']=lang
gr_data

,Genere,Titolo,ISBN,Rev_Date_GR,Rating_GR,Userlist_GR,N_Likes_GR,Shelves,Rev_Content_GR,Language_GR
0,Thriller,Triptych,9780385339469,"Oct 09, 2021",4,Carolyn Taylor,0,[],Looking to be another (older) but good series ...,en
1,Thriller,Triptych,9780385339469,"Oct 07, 2021",5,soperedi,0,[],"Good thrillerI very liked this book, fascinati...",en
2,Thriller,Triptych,9780385339469,"Oct 04, 2021",4,Ellen,0,"[library, audiobook]",I’ve been wanting to start this series for som...,en
3,Thriller,Triptych,9780385339469,"Oct 04, 2021",5,Harry Fuhrman,0,[],good read hooks you,en
4,Thriller,Triptych,9780385339469,"Oct 04, 2021",2,Yumi R,0,[],Predictable. Hoping the storyline would pick u...,en
...,...,...,...,...,...,...,...,...,...,...
31787,Scify,Golden Son,9780345539823,"Jul 02, 2021",5,Caleb Barker,0,[],Wow just really elevated what was established ...,en
31788,Scify,Golden Son,9780345539823,"Jul 02, 2021",4,Jolien,2,[],This book became better and better the further...,en
31789,Scify,Golden Son,9780345539823,"Jul 02, 2021",3,Kayla K,0,[],"I liked this book better than the first one, b...",en
31790,Scify,Golden Son,9780345539823,"Jul 01, 2021",5,Farid Yuzma Wiarco,0,[own],Gorydamn packed,cy


In [46]:
gr_data=gr_data[gr_data['Language_GR'] == 'en']
gr_data=gr_data.reset_index()
gr_data

,index,Genere,Titolo,ISBN,Rev_Date_GR,Rating_GR,Userlist_GR,N_Likes_GR,Shelves,Rev_Content_GR,Language_GR
0,0,Thriller,Triptych,9780385339469,"Oct 09, 2021",4,Carolyn Taylor,0,[],Looking to be another (older) but good series ...,en
1,1,Thriller,Triptych,9780385339469,"Oct 07, 2021",5,soperedi,0,[],"Good thrillerI very liked this book, fascinati...",en
2,2,Thriller,Triptych,9780385339469,"Oct 04, 2021",4,Ellen,0,"[library, audiobook]",I’ve been wanting to start this series for som...,en
3,3,Thriller,Triptych,9780385339469,"Oct 04, 2021",5,Harry Fuhrman,0,[],good read hooks you,en
4,4,Thriller,Triptych,9780385339469,"Oct 04, 2021",2,Yumi R,0,[],Predictable. Hoping the storyline would pick u...,en
...,...,...,...,...,...,...,...,...,...,...,...
30593,31786,Scify,Golden Son,9780345539823,"Jul 02, 2021",5,Emma Cathryne,6,"[romance-element, adult, sci-fi, very-dark]",I am annihilated by this book. I am undone by ...,en
30594,31787,Scify,Golden Son,9780345539823,"Jul 02, 2021",5,Caleb Barker,0,[],Wow just really elevated what was established ...,en
30595,31788,Scify,Golden Son,9780345539823,"Jul 02, 2021",4,Jolien,2,[],This book became better and better the further...,en
30596,31789,Scify,Golden Son,9780345539823,"Jul 02, 2021",3,Kayla K,0,[],"I liked this book better than the first one, b...",en


In [36]:
50000/250

200.0

In [47]:
gr_data.to_csv('goodreads_ant_705_850.csv')

In [ ]:
#il file lo trovi su one drive sezione codice, spostalo in locale nella cartella english
#poi sposta in locale questo e Yes24

In [52]:
single_title=[]
for el in titoli:
    if el not in single_title:
        single_title.append(el)
    else:
        continue
print(len(single_title))
print(single_title[:10])

144
['Triptych', 'No Country for Old Men', 'Child 44', 'Doctor Sleep', 'Mystic River', 'End of Watch', 'The Hard Way', 'Gone Tomorrow', 'Blindsighted', 'Echo Burning']
